# SITCOM-818
Craig Lage - 15-May-23 \

Here is what was requested:
A script is needed to return maximal overshoot values per M1M3 during bump testing. It shall retrieve FA following error (M1M3.forceActuatorData.primaryCylinderFollowingError and M1M3.forceActuatorData.secondaryCylinderFollowingError) and per actuator, while it is bump tested, retrieve min and max (absolute) value of the deviation.

This notebook does those things


## Prepare the notebook

In [ ]:
# Directory to store the data
data_dir = "/home/craiglagegit/DATA"

start = "2023-06-14T00:00:00"
end = "2023-06-15T00:00:00"

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline

import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable
from lsst_efd_client import EfdClient

In [ ]:
async def plot_bump_test_following_errors(fig, bumps, id):
    """ Plot a visualization of the bump test following errors
        Parameters
        ----------
        fig : a matplotlib figure object

        bumps: pandas dataframe
            This is a dataframe containg the bump test status
        
        id: 'int'
            The actuator id desired

        Returns
        -------
        No return, only the fig object which was input
    """
    
    this_bump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primary_bump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primary_follow = f"primaryCylinderFollowingError{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primary_force = f"primaryCylinderForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        secondary_bump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondary_force = f"secondaryCylinderForce{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondary_follow = f"secondaryCylinderFollowingError{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondary_name = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondary_name = None
        secondary_force = None
        secondary_follow = None
    plt.subplots_adjust(wspace=0.3)
    plt.suptitle(f"Bump Test Following Errors. Actuator ID {id}", fontsize=18)
    plot_start = this_bump[this_bump[primary_bump]==2]['timestamp'].values[0] - 1.0
    plot_end = plot_start + 14.0 
    start = Time(plot_start, format='unix_tai', scale='tai')
    end = Time(plot_end, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                             [primary_force, primary_follow, 'timestamp'], start.utc, end.utc)
    timestamp = forces.index[0].isoformat().split('.')[0]
    plt.suptitle(f"Bump Test Following Errors. Actuator ID {id}\n {timestamp}", fontsize=18)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    primary_forces = forces[primary_force].values
    primary_errors = forces[primary_follow].values
    plot_start -= t0
    plot_end -= t0
    plt.subplot(2,2,1)
    plt.title("Primary - Z")
    plt.plot(times, primary_forces, label='Data')
    plt.xlim(plot_start, plot_end)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
    plt.legend()
    plt.subplot(2,2,3)
    plt.title("Following Errors")
    plt.plot(times, primary_errors)
    plt.text(5, 350, f"Max = {np.max(primary_errors):.1f} nt")
    plt.text(5, -350, f"Min = {np.min(primary_errors):.1f} nt")
    plt.xlim(plot_start, plot_end)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Following Errors (nt)")
    
    if secondary_name is not None:
        plot_start = this_bump[this_bump[secondary_bump]==2]['timestamp'].values[0] - 1.0
        plot_end = plot_start + 14.0
        start = Time(plot_start, format='unix_tai', scale='tai')
        end = Time(plot_end, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                                 [secondary_force, secondary_follow, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        secondary_forces = forces[secondary_force].values
        secondary_errors = forces[secondary_follow].values
        plot_start -= t0
        plot_end -= t0
        plt.subplot(2,2,2)
        plt.title(f"Secondary - {secondary_name}")
        plt.plot(times, secondary_forces, label='Data')
        plt.xlim(plot_start, plot_end)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (nt)")
        plt.legend()
        plt.subplot(2,2,4)
        plt.title("Following Errors")
        plt.plot(times, secondary_errors)
        plt.text(5, 350, f"Max = {np.max(secondary_errors):.1f} nt")
        plt.text(5, -350, f"Min = {np.min(secondary_errors):.1f} nt")

        plt.xlim(plot_start, plot_end)
        plt.ylim(-400, 400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Following Errors (nt)")
    else:
        plt.subplot(2,2,2)
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
        plt.subplot(2,2,4)
        plt.xticks([])
        plt.yticks([])
    return

## First run just one actuator

In [ ]:
client = EfdClient('summit_efd')
FATABLE = M1M3FATable.FATABLE
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*",\
                                        Time(start, scale='utc'), Time(end, scale='utc'))
id = 112
fig = plt.figure(figsize=(10,10))
await plot_bump_test_following_errors(fig, bumps, id)
plt.savefig(data_dir + f"/Bump_Test_Following_Errors_{id}.png")

# Now run the whole bump test

In [ ]:
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
pdf = PdfPages(data_dir + f"/Bump_Test_Following_Errors_{timestamp}.pdf")
fig = plt.figure(figsize=(10,10))
for index in range(len(FATABLE)):
    try:
        id = FATABLE[index][M1M3FATable.FATABLE_ID]
        print(index, id)
        await plot_bump_test_following_errors(fig, bumps, id)
        pdf.savefig(fig)  # saves the current figure into a pdf page
        plt.clf()
    except:
        print(f"{id} failed!")
        continue
pdf.close()
